# **재정정보 AI 검색 알고리즘 경진대회 - RAG, **
> **RAG, **

본 대회의 과제는 중앙정부 재정 정보에 대한 **검색 기능**을 개선하고 활용도를 높이는 질의응답 알고리즘을 개발하는 것입니다. <br>이를 통해 방대한 재정 데이터를 일반 국민과 전문가 모두가 쉽게 접근하고 활용할 수 있도록 하는 것이 목표입니다. <br><br>
베이스라인에서는 평가 데이터셋만을 활용하여 source pdf 마다 Vector DB를 구축한 뒤 langchain 라이브러리와 llama-2-ko-7b 모델을 사용하여 RAG 프로세스를 통해 추론하는 과정을 담고 있습니다. <br>( train_set을 활용한 훈련 과정은 포함하지 않으며, test_set  에 대한 추론만 진행합니다. )

---

💡 **NOTE**: 이 예제에서 사용한 모델 및 벡터 DB 

1. LLM : mindsignal/rtzr-ko-gemma-2-9b-it-4bit-financesinfo-ver1
2. Embed Model : intfloat/multilingual-e5-large
3. Vector DB : FAISS

---

# Install

In [1]:

# 모델 가속화 및 메모리 관리
%pip install accelerate
%pip install -i https://pypi.org/simple/ bitsandbytes

# Transformer 기반 모델과 데이터셋 관련 라이브러리
%pip install transformers[torch] -U
%pip install datasets

# Llama-Index
%pip install llama-index
%pip install llama-index-llms-huggingface
%pip install llama-index-embeddings-huggingface
%pip install llama-index-vector-stores-faiss
%pip install llama-index-postprocessor-cohere-rerank
%pip install llama-index-readers-file pymupdf


# 벡터 데이터베이스 관련 라이브러리
%pip install faiss-cpu

# 데이터
%pip install scikit-learn
%pip install tiktoken

%load_ext autoreload
%autoreload 2

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 57.6/57.6 kB 803.7 kB/s eta 0:00:00 0:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 315.1/315.1 kB 2.7 MB/s eta 0:00:00a 0:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 417.5/417.5 kB 7.2 MB/s eta 0:00:00ta 0:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 435.5/435.5 kB 29.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 78.4/78.4 kB 17.4 MB/s eta 0:00:00

[notice] A new release of pip is available: 24.0 -> 24.2
[notice] To update, run: python -m pip install --upgrade pip
Note: you may need to restart the kernel to use updated packages.
Looking in indexes: https://pypi.org/simple/
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 137.5/137.5 MB 28.5 MB/s eta 0:00:0000:0100:01

[notice] A new release of pip is available: 24.0 -> 24.2
[notice] To update, run: python -m pip install --upgrade pip
Note: you may need to restart the kernel to use updated packages.
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 43.7/43.7 kB 553.7 kB

# Import Library

In [3]:
%pip install matplotlib

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 162.6/162.6 kB 1.1 MB/s eta 0:00:00a 0:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 8.3/8.3 MB 23.6 MB/s eta 0:00:00a 0:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 305.2/305.2 kB 673.5 kB/s eta 0:00:00a 0:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 4.6/4.6 MB 48.5 MB/s eta 0:00:00a 0:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.6/1.6 MB 70.6 MB/s eta 0:00:00

[notice] A new release of pip is available: 24.0 -> 24.2
[notice] To update, run: python -m pip install --upgrade pip
Note: you may need to restart the kernel to use updated packages.


In [4]:
import os
import re
import sys
import json
import torch
import logging
import unicodedata
# import numpy as np
import pandas as pd
import nest_asyncio
from tqdm import tqdm
from pathlib import Path

# from typing import Dict, List, Optional, Tuple

# import pymupdf4llm
# 
# AutoModelForCausalLM,
# pipeline,
# Gemma2ForCausalLM,
# TextStreamer,
    
from transformers import BitsAndBytesConfig, AutoTokenizer
# from accelerate import Accelerator

# Llama-Index

# 
# SimpleDirectoryReader,     
# ChatPromptTemplate,
# get_response_synthesizer,
# DocumentSummaryIndex,
# SummaryIndex
from llama_index.core import (
    StorageContext,
    VectorStoreIndex, 
    Settings,
    PromptTemplate,
    Document,
)
# from llama_index.core.response.notebook_utils import display_response
# from llama_index.core.response_synthesizers import TreeSummarize, Refine
from llama_index.llms.huggingface import HuggingFaceLLM
from llama_index.embeddings.huggingface import HuggingFaceEmbedding
from llama_index.core import Settings
from llama_index.readers.file import PyMuPDFReader
from llama_index.core.node_parser import SentenceSplitter
from llama_index.core.schema import TextNode
from llama_index.vector_stores.faiss import FaissVectorStore
import faiss
import pickle
from llama_index.core.response_synthesizers import TreeSummarize
from llama_index.core import StorageContext
from llama_index.core import SimpleKeywordTableIndex, VectorStoreIndex
from llama_index.core.tools import QueryEngineTool
from llama_index.core.query_engine import RouterQueryEngine
from llama_index.core.selectors import LLMMultiSelector
from llama_index.core import PromptTemplate
from llama_index.core import SummaryIndex
from llama_index.core.response.notebook_utils import display_response


# Setup

In [7]:
# 비동기 이벤트 루프를 재설정하거나 중첩할 수 있도록 허용
nest_asyncio.apply()

# 로깅 설정
logging.basicConfig(stream=sys.stdout, level=logging.INFO)
logger = logging.getLogger(__name__)

# CUDA 메모리 할당이 필요할 때마다 메모리 블록을 확장
os.environ["PYTORCH_CUDA_ALLOC_CONF"] = "expandable_segments:True"

In [8]:
class Config:
    def __init__(self, llm_name: str, embedding_name: str, concept: str) -> None:        
        # 모델별 설정 딕셔너리
        self.model_config = {
            "meta-llama/Meta-Llama-3.1-8B-Instruct": {
                "quantization_config": None,
                "torch_dtype": "auto",
                "max_token": 256,
            },
            "rtzr/ko-gemma-2-9b-it": {
                "quantization_config": self.get_quantization_config(),
                "torch_dtype": "auto",
                "max_token": 450,
            },
            "jjjguz/Llama-3.1-Korean-8B-Instruct-v1": {
                "quantization_config": None,
                "torch_dtype": "auto",
                "max_token": 256,
            },
            "mindsignal/rtzr-ko-gemma-2-9b-it-4bit-financesinfo-ver1": {
                "quantization_config": self.get_quantization_config(),
                "torch_dtype": "auto",
                "max_token": 450,
            },
            "mindsignal/upstage-SOLAR-10.7B-Instruct-v1.0-4bit-financesinfo-ver1": {
                "quantization_config": self.get_quantization_config(),
                "torch_dtype": torch.float16,
                "max_token": 512
            }
        }
        
        # Large Language Model
        self.llm_name = llm_name
        self.llm_config = self.model_config[self.llm_name]
        
        # 토크나이저 로드 및 설정
        tokenizer = AutoTokenizer.from_pretrained(self.llm_name)
        tokenizer.use_default_system_prompt = False
        
        Settings.llm = HuggingFaceLLM(
            model_name = self.llm_name,
            tokenizer= tokenizer,
            context_window=4096,# Solar 입력으로 처리할 최대 토큰 수
            max_new_tokens=4096, # 생성가능한 최대 토큰 수
            model_kwargs={
                "trust_remote_code": True, # 리모트 코드 실행을 허용
                "torch_dtype": self.llm_config["torch_dtype"], # 양자화된 dtype 설정
                "quantization_config": self.llm_config["quantization_config"], # 양자화 설정 적용
            },
            generate_kwargs={"temperature":0, "top_k":50, "top_p":0.95},
            device_map="auto",
            is_chat_model=False, # 채팅 모델이 아닌 일반 텍스트 생성 모델로 설정
        )

        # Embed Model
        Settings.embed_model = HuggingFaceEmbedding(
            model_name=embedding_name,
            device="cuda",
            normalize=True,
        )
        
        # Data Path
        self.base_directory = "open/"
        self.test_csv_path = os.path.join(self.base_directory, "test.csv")
        
        # Output Path
        self.output_dir = "test_results"
        self.output_csv_file = f"{self.llm_name.split('/')[1]}_{concept}_submission.csv"
        os.makedirs(self.output_dir, exist_ok=True)
    
    def to_json(self):
        return json.dumps(self.__dict__)
    
    def get_quantization_config(self):
        """4-bit 양자화 설정을 반환하는 함수"""
        return BitsAndBytesConfig(
            load_in_4bit=True,  # 4-bit 양자화
            bnb_4bit_use_double_quant=True,
            bnb_4bit_quant_type="nf4",
            bnb_4bit_compute_dtype=torch.bfloat16  # 연산에 사용할 데이터 타입
        )
        

# PDF -> Vector Store

In [14]:

class DataIngestionPipeline:
    def __init__(self, config:Config, chunk_sizes:list):
        self.embed_model = Settings.embed_model
        self.config = config
        self.chunk_sizes = chunk_sizes # [128, 256, 512, 1024]
        
    def normalize_path(self, path):
        """ Path 유니코드 정규화 """
        normalized_path = unicodedata.normalize('NFD', path)
        logger.debug(f"정규화된 경로: {normalized_path}")
        return normalized_path
    
    def preprocess_text(self, text: str) -> str:
        """ 텍스트 전처리 함수 """
        # 불필요한 공백 제거
        text = text.strip()
        # 마침표 뒤 및 "----" 전후의 줄바꿈을 제외한 모든 줄바꿈을 제거
        text = re.sub(r'(?<!\.)(?<!-----)(\n|\r\n)(?!-----)', ' ', text)
        return text
    
    def create_nodes_from_pdf(self, file_path: str, chunk_sizes) -> list:
        """ PDF 파일 로드, 텍스트 추출 및 전처리 후 노드 생성 """
        logger.info(f"PDF -> nodes : {file_path}")
        
        # Step 1: PDF 파일 로드 및 텍스트 추출
        loader = PyMuPDFReader()
        docs0 = loader.load(file_path=Path(file_path))
        doc_text = "\n\n".join([d.get_content() for d in docs0]) # 전체 페이지 텍스트 병합
        
        # Step 2: 텍스트 전처리
        processed_text = self.preprocess_text(doc_text)
        docs = [Document(text=processed_text)]
        
        # Step 3: 문서를 청크로 분할
        nodes_list = []
        vector_indices = []
        for chunk_size in chunk_sizes:
            print(f"Chunk Size: {chunk_size}")
            splitter = SentenceSplitter(chunk_size=chunk_size, chunk_overlap=50)
            nodes = splitter.get_nodes_from_documents(docs)
            
            # 나중에 추적할 노드에 청크 크기 추가
            for node in nodes:
                node.metadata["chunk_size"] = chunk_size
                node.excluded_embed_metadata_keys = ["chunk_size"]
                node.excluded_llm_metadata_keys = ["chunk_size"]
            
            nodes_list.append(nodes)
            
            # build vector index
            vector_index = VectorStoreIndex(nodes)
            vector_indices.append(vector_index)

        return nodes_list, vector_indices
    
    
    def init_vector_db(self, df):
        """ 통합 실행 함수 : PDF 데이터셋으로부터 인덱스 구축 """
        vector_databases = {}
        unique_paths = df['Source_path'].unique()
        chunk_sizes = self.chunk_sizes
        
        for path in tqdm(unique_paths, desc="PDFs -> Index"):
            # 각 pdf 파일에 대한 경로 정규화 및 절대 경로 생성
            normalized_path = self.normalize_path(path)
            full_path = os.path.normpath(os.path.join(self.config.base_directory, normalized_path.lstrip('./'))) if not os.path.isabs(normalized_path) else normalized_path

            # pdf 제목 추출
            pdf_title = os.path.splitext(os.path.basename(full_path))[0]
            logger.info(f'처리 중 : {pdf_title}...')
            
            # PDF 로드, 텍스트 추출 및 전처리, 청크로 분할 후 노드 생성
            nodes_list, vector_indices = self.create_nodes_from_pdf(full_path, chunk_sizes)

            # # FAISS 인덱스 생성
            # d = 1024 # intfloat/multilingual-e5-large 모델의 임베딩 차원 수
            # faiss_index = faiss.IndexFlatL2(d)
            
            # # FAISS 벡터 스토어 생성
            # vector_store = FaissVectorStore(faiss_index=faiss_index)
            # storage_context = StorageContext.from_defaults(vector_store=vector_store)
            
            # 각 PDF에 대해 nodes_list와 vector_indices를 저장
            vector_databases[pdf_title] = {
                'nodes_list': nodes_list,
                'vector_indices': vector_indices
            }
            
        logger.info("모든 PDF -> vector_store 처리 완료.")
        return vector_databases
            


# QA 생성 모듈

In [10]:


class Prompts:
    def __init__(self):
        self.qa_prompt = self.get_qa_prompt()
        # self.tree_summarize_prompt = self.get_tree_summarizer()
        
    def get_qa_prompt(self):
        """ QA 프롬프트 템플릿 """
        QA_PROMPT_TMPL = (
            """ 
            당신은 중앙정부 재정 정책 전문가입니다. 주어진 재정 정보를 바탕으로, 명확하고 정확한 답변을 제시하세요.\n
            주어진 컨텍스트는 다음과 같습니다.\n
            ---------------------\n
            {context_str}\n
            ---------------------\n

            주어진 질문에 대해 다음 단계에 따라 생각한 후 답변하세요:\n
            1. 질문을 이해하고 필요한 재정 개념을 설명합니다.\n
            2. 질문을 단계별로 분석하여 논리적인 답변을 구성합니다.\n
            3. 관련 개념 간의 관계를 분석하여 맥락을 명확히 합니다.\n

            답변을 생성할 때에는 "Response:", "AI:" 등과 같은 불필요한 텍스트를 포함하지 말고, 핵심 정보만 명확하게 전달하세요.\n
            답변은 간결하고 명확하게 작성하며, 주어진 질문에 대해 핵심만 파악하여 답변하세요.\n
            단, 주어와 서술어를 사용하여 온전한 문장을 완성시켜 최대한 자연스럽게 답변해야 합니다.\n

            주어진 질문은 다음과 같습니다. : {query_str}\n
            답변 :\n
            """
        )
        return PromptTemplate(QA_PROMPT_TMPL)
    
    # def get_tree_summarizer(self):
    #     """ TreeSummarize 생성 """
    #     TREE_SUMMARIZE_PROMPT_TMPL = ("""
    #         여러 출처의 컨텍스트 정보가 아래에 나와 있습니다.\n
    #         각 소스에는 relevance score가 첨부되어 있을 수도 있고 없을 수도 있습니다.\n
    #         ---------------------\n
    #         {context_str}\n
    #         ---------------------\n
    #         사전 지식이 아닌 여러 출처의 컨텍스트 정보를 참고하여 질문에 답하세요.\n
    #         주어진 질문은 다음과 같습니다. : {query_str}\n
    #         답변 :\n            
    #         """
    #     )
    #     return TreeSummarize(
    #         summary_template=PromptTemplate(TREE_SUMMARIZE_PROMPT_TMPL)
    #     )

In [20]:
import asyncio
from llama_index.core.retrievers import RouterRetriever, RecursiveRetriever
from llama_index.core import SummaryIndex
from llama_index.core.query_engine import RetrieverQueryEngine
from llama_index.core.schema import IndexNode
from llama_index.core.tools import RetrieverTool

from llama_index.core.selectors import PydanticMultiSelector


from llama_index.core.postprocessor import LLMRerank, SentenceTransformerRerank
# from llama_index.postprocessor.cohere_rerank import CohereRerank

from llama_index.core.response.notebook_utils import display_response


class FISearch:
    def __init__(self, config: Config, chunk_sizes:list) -> None:
        self.config = config
        self.llm_obj = Settings.llm
        self.prompts = Prompts()
        self.chunk_sizes = chunk_sizes
        
    def normalize_string(self, s):
        """ 유니코드 정규화 """
        return unicodedata.normalize('NFC', s)
        
    def post_processing(self, answer):
        """ 후처리 함수 """
        # 다양한 불필요한 텍스트 패턴을 제거
        patterns_to_remove = [
            r"^\s*###? Response:\s*",
            r"^\s*Response:\s*",
            r"^\s*AI:\s*",
            r"^\s*Quad:\s*",
            r"^\s*\|\-\s*\n?\s*\n?\s*###? Response:\s*",
            r"^\s*\|\-\s*\n?\s*\n?\s*Response:\s*",
            r"^\s*\|\-\s*\n?\s*\n?\s*AI:\s*",
            r"^\s*\|\-\s*\n?\s*\n?\s*Quad:\s*",
            r"<eos>\s*$"
        ]
        for pattern in patterns_to_remove:
            answer = re.sub(pattern, "", answer)
        return answer.strip()
    
    
    def generate_answer(self, df, vector_databases):
        """ DataFrame의 각 질문에 대한 답변 생성 후 저장 """
        results = []
        
        for _, row in tqdm(df.iterrows(), total=len(df), desc="Answering Questions"):
            source = self.normalize_string(row['Source'])
            query_str = row['Question']
            
            # 정규화된 키로 데이터베이스 검색
            normalized_keys = {self.normalize_string(k): v for k, v in vector_databases.items()}
            pdf_data = normalized_keys[source]
            vector_indices = pdf_data['vector_indices']
            
            # 앙상블 리트리버 생성
            # retriever_tools = []
            retriever_dict = {}
            retriever_nodes = []
            
            for chunk_size, vector_index in zip(self.chunk_sizes, vector_indices):
                node_id = f"chunk_{chunk_size}"
                node = IndexNode(
                    text=(f"중앙 정부 재정 정책 문서에서 관련 컨텍스트를 검색합니다.(청크 크기: {chunk_size})"),
                    index_id = node_id,
                )
                retriever_nodes.append(node)
                retriever_dict[node_id] = vector_index.as_retriever()

            # Recursive retriever 정의
            # 파생된 리트리버는 모든 노드를 검색합니다.
            summary_index = SummaryIndex(retriever_nodes)
            retriever = RecursiveRetriever(
                root_id="root",
                retriever_dict={"root": summary_index.as_retriever(), **retriever_dict},
            )
        
            # 리랭커 정의
            # reranker = LLMRerank()
            reranker = SentenceTransformerRerank(top_n=10)
            # reranker = CohereRerank(top_n=10)
            
            # 리트리버 쿼리 엔진 정의
            query_engine = RetrieverQueryEngine(retriever, node_postprocessors=[reranker])
        
            # 검색 
            response = query_engine.query(query_str)
            
            # QA 프롬프트 생성 및 적용
            # prompt = self.prompts.qa_prompt.format(context_str=context, query_str=query_str)
            # response = self.llm_obj.generate(prompt)
            
            display_response(
                response, show_source=True, source_length=500, show_source_metadata=True
            )
            
            logger.info('llm 생성 결과 : \n', response)
            processed_response = self.post_processing(response) # 후처리 적용
            logger.info('후처리 결과 : \n', processed_response)
            
            # 결과 저장
            results.append({
                "Source": row['Source'],
                "Source_path": row['Source_path'],
                "Question": query_str,
                "Answer": processed_response
            })
            
        return results
    

---

# 최종 통합, 실행 및 저장

In [ ]:
import asyncio

if __name__ == "__main__":
    # 초기 세팅
    config = Config(
        llm_name="mindsignal/rtzr-ko-gemma-2-9b-it-4bit-financesinfo-ver1", 
        embedding_name="intfloat/multilingual-e5-large",
        concept="financial_analysis"
    )
    chunk_sizes = [128, 256, 512, 1024]
    
    # 데이터 불러오기
    df = pd.read_csv(config.test_csv_path)
    
    # 파이프라인 초기화
    pipeline = DataIngestionPipeline(config, chunk_sizes)
    
    # 벡터 데이터베이스 초기화 - 메모리에 저장
    vector_databases = pipeline.init_vector_db(df)
    
    # 검색 및 답변 생성
    search_engine = FISearch(config, chunk_sizes)
    results = search_engine.generate_answer(df, vector_databases)
    
    # 결과를 제출 양식에 맞게 저장
    submit_df = pd.read_csv(f"./open/sample_submission.csv")
    submit_df['Answer'] = [item['Answer'] for item in results]
    submit_df['Answer'] = submit_df['Answer'].fillna("데이콘")
    
    # 결과를 CSV 파일로 저장
    submit_df.to_csv(os.path.join(config.output_dir, config.output_csv_file), encoding='UTF-8-sig', index=False)
    
    print("결과가 성공적으로 저장되었습니다.")

tokenizer_config.json:   0%|          | 0.00/40.5k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/17.5M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/636 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/895 [00:00<?, ?B/s]

model.safetensors.index.json:   0%|          | 0.00/39.1k [00:00<?, ?B/s]

model-00001-of-00004.safetensors:   0%|          | 0.00/4.90G [00:00<?, ?B/s]

model-00002-of-00004.safetensors:   0%|          | 0.00/4.95G [00:00<?, ?B/s]

model-00003-of-00004.safetensors:   0%|          | 0.00/4.96G [00:00<?, ?B/s]

model-00004-of-00004.safetensors:   0%|          | 0.00/3.67G [00:00<?, ?B/s]